In [23]:
import pandas as pd
from ucimlrepo import fetch_ucirepo
from sdv.single_table import GaussianCopulaSynthesizer, CTGANSynthesizer
from sdv.evaluation.single_table import run_diagnostic, evaluate_quality, get_column_plot, get_column_pair_plot
from sdv.utils import load_synthesizer
from sdv.metadata import SingleTableMetadata
import os
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
adult = fetch_ucirepo(id=2) # Adult dataset

In [3]:
adult_df = pd.concat([adult.data.features, adult.data.targets], axis=1)
adult_df['income'] = adult_df['income'].str.strip().str.replace('.', '', regex=False)
metadata = SingleTableMetadata()
metadata.detect_from_dataframe(data=adult_df)
print("Dataset loaded successfully.")

Dataset loaded successfully.


In [4]:
adult_df.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [5]:
print(adult_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48842 entries, 0 to 48841
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             48842 non-null  int64 
 1   workclass       47879 non-null  object
 2   fnlwgt          48842 non-null  int64 
 3   education       48842 non-null  object
 4   education-num   48842 non-null  int64 
 5   marital-status  48842 non-null  object
 6   occupation      47876 non-null  object
 7   relationship    48842 non-null  object
 8   race            48842 non-null  object
 9   sex             48842 non-null  object
 10  capital-gain    48842 non-null  int64 
 11  capital-loss    48842 non-null  int64 
 12  hours-per-week  48842 non-null  int64 
 13  native-country  48568 non-null  object
 14  income          48842 non-null  object
dtypes: int64(6), object(9)
memory usage: 5.6+ MB
None


In [6]:
print(adult_df.shape)

(48842, 15)


In [7]:

for column in adult_df.select_dtypes(include=['object']).columns:
    print(f"\nValue counts for column '{column}':")
    print(adult_df[column].value_counts())


Value counts for column 'workclass':
workclass
Private             33906
Self-emp-not-inc     3862
Local-gov            3136
State-gov            1981
?                    1836
Self-emp-inc         1695
Federal-gov          1432
Without-pay            21
Never-worked           10
Name: count, dtype: int64

Value counts for column 'education':
education
HS-grad         15784
Some-college    10878
Bachelors        8025
Masters          2657
Assoc-voc        2061
11th             1812
Assoc-acdm       1601
10th             1389
7th-8th           955
Prof-school       834
9th               756
12th              657
Doctorate         594
5th-6th           509
1st-4th           247
Preschool          83
Name: count, dtype: int64

Value counts for column 'marital-status':
marital-status
Married-civ-spouse       22379
Never-married            16117
Divorced                  6633
Separated                 1530
Widowed                   1518
Married-spouse-absent      628
Married-AF-spouse     

In [9]:
synthesizer = CTGANSynthesizer(metadata, verbose=True)
synthesizer.fit(adult_df)

/home/gabriel/Synthetic_data_generation/.venv/lib/python3.11/site-packages/sdv/single_table/base.py:168: FutureWarning: The 'SingleTableMetadata' is deprecated. Please use the new 'Metadata' class for synthesizers.
  warnings.warn(DEPRECATION_MSG, FutureWarning)
Gen. (-0.51) | Discrim. (-0.06): 100%|██████████| 300/300 [25:02<00:00,  5.01s/it]


In [10]:
#save model
model_path = './models/CTGAN/1/synthesizer.pkl'
os.makedirs(os.path.dirname(model_path), exist_ok=True)
synthesizer.save(model_path)
print("Model trained and saved successfully.")

Model trained and saved successfully.


In [ ]:
#load model
# model_path = './models/CTGAN/1/synthesizer.pkl'
# synthesizer = load_synthesizer(model_path)
# print("Model loaded successfully.")

In [11]:
synthetic_data = synthesizer.sample(num_rows=len(adult_df))


print("Synthetic data generated successfully!")
print(synthetic_data.head())

Synthetic data generated successfully!
   age     workclass  fnlwgt     education  education-num      marital-status  \
0   36  Self-emp-inc  239820     Bachelors             13       Never-married   
1   24       Private  189623     Bachelors             13       Never-married   
2   24       Private  310224  Some-college             10       Never-married   
3   31   Federal-gov  173321  Some-college             10  Married-civ-spouse   
4   27  Self-emp-inc  112444     Bachelors             13            Divorced   

        occupation   relationship   race     sex  capital-gain  capital-loss  \
0  Protective-serv        Husband  White    Male             7             0   
1   Prof-specialty  Not-in-family  White  Female             9             0   
2     Adm-clerical      Own-child  White    Male             0             0   
3  Protective-serv        Husband  White    Male             9             0   
4                ?      Unmarried  White  Female          2824            

In [12]:
diagnostic_report = run_diagnostic(
    real_data=adult_df,
    synthetic_data=synthetic_data,
    metadata=metadata
)

print("--- Diagnostic Report ---")
print(diagnostic_report)

Generating report ...

(1/2) Evaluating Data Validity: |██████████| 15/15 [00:00<00:00, 124.57it/s]|
Data Validity Score: 100.0%

(2/2) Evaluating Data Structure: |██████████| 1/1 [00:00<00:00, 258.88it/s]|
Data Structure Score: 100.0%

Overall Score (Average): 100.0%

--- Diagnostic Report ---


In [13]:
quality_report = evaluate_quality(
    real_data=adult_df,
    synthetic_data=synthetic_data,
    metadata=metadata
)

print("\n--- Full Quality Report Summary ---")
print(quality_report)

Generating report ...

(1/2) Evaluating Column Shapes: |██████████| 15/15 [00:00<00:00, 53.04it/s]|
Column Shapes Score: 84.86%

(2/2) Evaluating Column Pair Trends: |██████████| 105/105 [00:02<00:00, 47.22it/s]|
Column Pair Trends Score: 79.17%

Overall Score (Average): 82.01%


--- Full Quality Report Summary ---


In [25]:
output_dir = './images/CTGAN/1/'
os.makedirs(output_dir, exist_ok=True)
print(f"Saving Matplotlib/Seaborn images to '{output_dir}' directory...")


column_names = adult_df.columns

for column in column_names:
    print(f"Generating plot for column: '{column}'...")
    plt.figure(figsize=(12, 7))

    if pd.api.types.is_numeric_dtype(adult_df[column]):
        sns.kdeplot(adult_df[column], label='Real', fill=True, alpha=0.5, warn_singular=False)
        sns.kdeplot(synthetic_data[column], label='Synthetic', fill=True, alpha=0.5, warn_singular=False)
        plt.title(f'Numerical Distribution: {column}', fontsize=16)

    else:
        # Combine data for proper categorical plotting with hue
        combined_df = pd.concat([
            adult_df[[column]].assign(Source='Real'),
            synthetic_data[[column]].assign(Source='Synthetic')
        ]).reset_index(drop=True)  # <-- THE FIX IS HERE

        sns.countplot(data=combined_df, x=column, hue='Source')
        plt.title(f'Categorical Distribution: {column}', fontsize=16)
        plt.xticks(rotation=45, ha='right')

    plt.legend()
    plt.tight_layout()
    
    file_path = os.path.join(output_dir, f'{column}.png')
    plt.savefig(file_path)
    
    plt.close()

print("\nAll plots have been saved successfully using Matplotlib/Seaborn.")

Saving Matplotlib/Seaborn images to './images/CTGAN/1/' directory...
Generating plot for column: 'age'...
Generating plot for column: 'workclass'...
Generating plot for column: 'fnlwgt'...
Generating plot for column: 'education'...
Generating plot for column: 'education-num'...
Generating plot for column: 'marital-status'...
Generating plot for column: 'occupation'...
Generating plot for column: 'relationship'...
Generating plot for column: 'race'...
Generating plot for column: 'sex'...
Generating plot for column: 'capital-gain'...
Generating plot for column: 'capital-loss'...
Generating plot for column: 'hours-per-week'...
Generating plot for column: 'native-country'...
Generating plot for column: 'income'...

All plots have been saved successfully using Matplotlib/Seaborn.
